# Step 1 - Install the required dependencies and make sure the python version is 3.10 and above

In [ ]:
!pip install zeno-client
!pip install --upgrade pip
!pip install --upgrade bottleneck
!pip install langdetect


In [7]:
!python --version

Python 3.10.12


In [ ]:
from zeno_client import ZenoClient, ZenoMetric
import pandas as pd

# Initialize a client with our API key.
client = ZenoClient("<INSERT YOUR API KEY>")


# Step 2 - Create a project

In [ ]:
project = client.create_project(
    name="Biased Movies Analysis",
    view="text-classification",
    metrics=[
        ZenoMetric(name="accuracy", type="mean", columns=["correct"]),
        #Uncomment the following line after implementing the function
        # ZenoMetric(name="avg length ratio", type="mean", columns=["avg_length_ratio"]),
    ]
)

# Step 3 - Create dataset

In [ ]:
import pandas as pd
from langdetect import detect
from collections import Counter
import random

# Sample language detection function
def detect_language(text):
    try:
        return detect(text)
    except:
        return "Unknown"

# Create a Pandas DataFrame for the biased dataset of movies
df = pd.DataFrame(
    {
        "id": range(1, 11),  # Using only 10 instances for demonstration
        "text": [
            "I love this movie!",
            "¡Odio esta película!",
            "Ce film est ok.",
            "The acting was great!",
            "The plot was confusing.",
            "I really enjoyed it!",
            "Worst movie ever!",
            "Not bad, but could be better.",
            "A masterpiece!",
            "I didn't like it.",
        ],
        "genre": ["action", "drama", "comedy", "action", "drama", "action", "drama", "comedy", "action", "drama"],
        "year": [random.choice([2020, 2021, 2022]) for _ in range(10)],  # Randomly select year
        "label": ["positive", "negative", "neutral", "positive", "negative", "positive", "negative", "neutral", "positive", "negative"],
    }
)

# Introduce biases in genre
biased_genre = ["action", "action", "drama", "drama", "comedy", "comedy", "comedy", "comedy", "drama", "drama"]
df["biased_genre"] = biased_genre

# Introduce biases in year
biased_year = [2020, 2020, 2020, 2021, 2021, 2021, 2022, 2022, 2022, 2022]
df["biased_year"] = biased_year

# Add language detection to create a new column
df["detected_language"] = df["text"].apply(detect_language)

# Add additional columns for analysis
df["input_length"] = df["text"].str.len()

#Uncomment the following line
# df["avg_length_ratio"] = {TBD}



# Step 4 - Upload the Base Dataset


In [ ]:

project.upload_dataset(df, id_column="id", data_column="text", label_column="label")



# Step 5 - Upload the AI System Outputs


In [ ]:
# For demonstration, let's assume we have system predictions
df_system = pd.DataFrame(
    {
        "output": ["positive", "negative", "negative", "positive", "negative", "positive", "negative", "neutral", "positive", "negative"],
    }
)

# Create an id column to match the base dataset
df_system["id"] = df_system.index + 1

# Measure accuracy for each instance, which is averaged by the ZenoMetric above
df_system["correct"] = (df_system["output"] == df["label"]).astype(int)



# Step 6 - Upload the system outputs


In [ ]:
project.upload_system(df_system, name="System A", id_column="id", output_column="output")